In [1]:
import sqlite3
import pandas as pd
import json
import numpy as np
import re
from tqdm.notebook import tqdm
tqdm.pandas()

import scipy
from scipy import stats
import statsmodels
import statsmodels.api as sm

from plotly.offline import init_notebook_mode, iplot
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import chart_studio
import chart_studio.plotly as py
from plotly import tools
username = 'JoaquinLCalvo' # your username
api_key = '98U3Yr92badRD9RKYXnS' # your api api_key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)
init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default='notebook'

import warnings
warnings.filterwarnings("ignore")

from perspective import PerspectiveAPI
from pysentimiento.preprocessing import preprocess_tweet

In [2]:
# Create your connection.
cnx = sqlite3.connect(r'C:\Users\jlopezca\Documents\TESI\TESIS-Cuando-el-odio-garpa\comments.db')
analyzed = pd.read_sql_query("SELECT * FROM comment", sqlite3.connect(r'C:\Users\jlopezca\Documents\TESI\TESIS-Cuando-el-odio-garpa\comments.db'))

In [4]:
analyzed.head(3)

,text,toxicity,json_response
0,😂pero si youtube no lleva ni 40 años jajaja,0.146869,"{""attributeScores"": {""THREAT_EXPERIMENTAL"": {""..."
1,"<a href=""https://www.youtube.com/watch?v=u6krw...",0.088079,"{""attributeScores"": {""THREAT_EXPERIMENTAL"": {""..."
2,Cj: te extrañe. Daniel Loquendo xd: y yo a ti cj,0.095093,"{""attributeScores"": {""TOXICITY"": {""spanScores""..."


In [6]:
analyzed.drop_duplicates(subset ='text', keep = False, inplace = True) 
analyzed.shape

(1280492, 3)

In [14]:
analyzed.dtypes

text              object
toxicity         float64
json_response     object
dtype: object

In [19]:
pd.set_option('max_colwidth', 8000)

In [20]:
analyzed.json_response.iloc[:1]

0    {"attributeScores": {"THREAT_EXPERIMENTAL": {"spanScores": [{"begin": 0, "end": 62, "score": {"value": 0.06943103, "type": "PROBABILITY"}}], "summaryScore": {"value": 0.06943103, "type": "PROBABILITY"}}, "IDENTITY_ATTACK_EXPERIMENTAL": {"spanScores": [{"begin": 0, "end": 62, "score": {"value": 0.074238, "type": "PROBABILITY"}}], "summaryScore": {"value": 0.074238, "type": "PROBABILITY"}}, "INSULT_EXPERIMENTAL": {"spanScores": [{"begin": 0, "end": 62, "score": {"value": 0.1857565, "type": "PROBABILITY"}}], "summaryScore": {"value": 0.1857565, "type": "PROBABILITY"}}, "TOXICITY": {"spanScores": [{"begin": 0, "end": 62, "score": {"value": 0.1468691, "type": "PROBABILITY"}}], "summaryScore": {"value": 0.1468691, "type": "PROBABILITY"}}, "PROFANITY_EXPERIMENTAL": {"spanScores": [{"begin": 0, "end": 62, "score": {"value": 0.07318536, "type": "PROBABILITY"}}], "summaryScore": {"value": 0.07318536, "type": "PROBABILITY"}}, "SEVERE_TOXICITY": {"spanScores": [{"begin": 0, "end": 62, "score"

In [22]:
df = pd.json_normalize(analyzed.to_dict(), 'json_response', ['THREAT_EXPERIMENTAL', 'IDENTITY_ATTACK_EXPERIMENTAL',
                                                   'INSULT_EXPERIMENTAL', 'PROFANITY_EXPERIMENTAL', 
                                                   'SEVERE_TOXICITY'], 
                    record_prefix='json_response_')
df.head()


MemoryError: 